In [53]:
import os
import time
import scipy
import netCDF4
import numpy as np

# Constants

In [54]:
paths = {}
paths["AICE"] = "/lustre/storeB/project/fou/hi/oper/aice/archive/"
paths["Barents"] = "/lustre/storeB/project/copernicus/cosi/PCAPS/Barents_AICE_grid/"
paths["ice_charts"] = "/lustre/storeB/project/copernicus/cosi/WP3/Operational/Ice_charts/"
paths["output"] = "/lustre/storeB/project/copernicus/cosi/AICE/Data/Distance_to_land/"

# Make common land sea mask

In [55]:
class make_common_land_sea_mask():
    def __init__(self, paths, date_task):
        self.paths = paths
        self.date_task = date_task

    def load_AICE(self):
        AICE = {}
        filename = self.paths["AICE"] + "AICE_forecasts_" + self.date_task + "T000000Z.nc"
        with netCDF4.Dataset(filename, "r") as nc:
            for var in nc.variables:
                AICE[var] = nc.variables[var][:]
        return AICE

    def load_Barents(self):
        Barents = {}
        filename = self.paths["Barents"] + self.date_task[0:4] + "/" + self.date_task[4:6] + "/" + "Barents_on_AICE_grid_" + self.date_task + ".nc"
        with netCDF4.Dataset(filename, "r") as nc:
            for var in nc.variables:
                Barents[var] = nc.variables[var][:]
        return Barents

    def load_ice_charts(self):
        Ice_charts = {}
        filename = self.paths["ice_charts"] + self.date_task[0:4] + "/" + self.date_task[4:6] + "/" + "Ice_charts_AICE_grid_" + self.date_task + ".nc"
        with netCDF4.Dataset(filename, "r") as nc:
            for var in nc.variables:
                Ice_charts[var] = nc.variables[var][:]
        return Ice_charts
    
    def make_land_sea_mask(self):
        AICE = self.load_AICE()
        Barents = self.load_Barents()
        Ice_charts = self.load_ice_charts()

        LSM_Barents = Barents["sea_mask"]
        LSM_AICE = np.ones(np.shape(AICE["SIC"][0,:,:]))
        LSM_AICE[np.isnan(AICE["SIC"][0,:,:]) == True] = 0
        LSM_ice_charts = np.ones(np.shape(Ice_charts["SIC"][0,:,:]))
        LSM_ice_charts[np.isnan(Ice_charts["SIC"][0,:,:]) == True] = 0

        LSM = np.zeros(np.shape(LSM_AICE))
        LSM[np.logical_and(LSM_ice_charts == 1, np.logical_and(LSM_Barents == 1, LSM_AICE == 1))] = 1

        return LSM

    def get_spatial_resolution(self, AICE):
        spatial_resolution = np.abs(AICE["x"][1] - AICE["x"][0])
        return spatial_resolution

    def calculate_distance_to_land(self, LSM, spatial_resolution):
        distance_to_land = scipy.ndimage.distance_transform_edt(LSM) * spatial_resolution
        return distance_to_land

    def __call__(self):
        AICE = self.load_AICE()
        LSM = self.make_land_sea_mask()
        spatial_resolution = self.get_spatial_resolution(AICE= AICE)
        distance_to_land = self.calculate_distance_to_land(LSM = LSM, spatial_resolution = spatial_resolution)
        return(AICE, LSM, distance_to_land)

# Write netCDF output

In [56]:
def write_netCDF(AICE, LSM, distance_to_land, paths):
    output_filename = paths["output"] + "AICE_LSM_distance_to_land.nc"
    if os.path.isfile(output_filename):
        os.system("rm " + output_filename)

    with netCDF4.Dataset(str(output_filename), "w", format = "NETCDF4") as output_netcdf:
        proj4 = output_netcdf.createDimension("proj4", 1)
        x = output_netcdf.createDimension("x", len(AICE["x"]))
        y = output_netcdf.createDimension("y", len(AICE["y"]))

        Outputs = vars()

        Outputs["proj4"] = output_netcdf.createVariable("proj4", "S1", ("proj4"))
        Outputs["proj4"].long_name = "+ellps=WGS84 +lat_0=90 +lon_0=0 +no_defs=None +proj=laea +type=crs +units=m +x_0=0 +y_0=0"

        Outputs["x"] = output_netcdf.createVariable("x", "d", ("x"))
        Outputs["x"][:] = np.copy(AICE["x"])
        Outputs["x"].units = "m"
        Outputs["x"].standard_name = "projection_x_coordinate"

        Outputs["y"] = output_netcdf.createVariable("y", "d", ("y"))
        Outputs["y"][:] = np.copy(AICE["y"])
        Outputs["y"].units = "m"
        Outputs["y"].standard_name = "projection_y_coordinate"

        Outputs["LSM"] = output_netcdf.createVariable("LSM", "d", ("y", "x"))
        Outputs["LSM"][:,:] = np.copy(LSM)
        Outputs["LSM"].units = "0: land, 1: ocean"
        Outputs["LSM"].standard_name = "Land sea mask"

        Outputs["distance_to_land"] = output_netcdf.createVariable("distance_to_land", "d", ("y", "x"))
        Outputs["distance_to_land"][:,:] = np.copy(distance_to_land)
        Outputs["distance_to_land"].units = "m"
        Outputs["distance_to_land"].standard_name = "Distance to the nearest land grid point"


# Main

In [57]:
t0 = time.time()
AICE, LSM, distance_to_land = make_common_land_sea_mask(paths = paths, date_task = "20250102")()
write_netCDF(AICE = AICE, LSM = LSM, distance_to_land = distance_to_land, paths = paths)
tf = time.time()
print("Computing time: ", tf - t0)

Computing time:  0.7889895439147949
